In [25]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import yfinance as yf
import tensorflow as tf
from datetime import datetime , timedelta
from tqdm import tqdm
from time import time
from ta.momentum import RSIIndicator
from sklearn.preprocessing import StandardScaler
import os
import warnings
import requests

scaler = StandardScaler()

#### Firstly we get the stickers for NIFTY50

In [26]:
from io import StringIO
url = 'https://en.wikipedia.org/wiki/NIFTY_50'

# Add headers to avoid 403 error
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}

# Fetch the page content with headers
response = requests.get(url, headers=headers)
table = pd.read_html(StringIO(response.text))

nifty50_df = table[1]  # Table 1 contains the NIFTY 50 companies
nifty50_tickers = nifty50_df['Symbol'].tolist()
nifty50_tickers = [ticker + '.NS' for ticker in nifty50_tickers]
print(nifty50_tickers)

['ADANIENT.NS', 'ADANIPORTS.NS', 'APOLLOHOSP.NS', 'ASIANPAINT.NS', 'AXISBANK.NS', 'BAJAJ-AUTO.NS', 'BAJFINANCE.NS', 'BAJAJFINSV.NS', 'BEL.NS', 'BHARTIARTL.NS', 'CIPLA.NS', 'COALINDIA.NS', 'DRREDDY.NS', 'EICHERMOT.NS', 'ETERNAL.NS', 'GRASIM.NS', 'HCLTECH.NS', 'HDFCBANK.NS', 'HDFCLIFE.NS', 'HINDALCO.NS', 'HINDUNILVR.NS', 'ICICIBANK.NS', 'INDIGO.NS', 'INFY.NS', 'ITC.NS', 'JIOFIN.NS', 'JSWSTEEL.NS', 'KOTAKBANK.NS', 'LT.NS', 'M&M.NS', 'MARUTI.NS', 'MAXHEALTH.NS', 'NESTLEIND.NS', 'NTPC.NS', 'ONGC.NS', 'POWERGRID.NS', 'RELIANCE.NS', 'SBILIFE.NS', 'SHRIRAMFIN.NS', 'SBIN.NS', 'SUNPHARMA.NS', 'TCS.NS', 'TATACONSUM.NS', 'TATAMOTORS.NS', 'TATASTEEL.NS', 'TECHM.NS', 'TITAN.NS', 'TRENT.NS', 'ULTRACEMCO.NS', 'WIPRO.NS']


#### Importing all the libraries required to build the LSTM

In [27]:
from tensorflow.keras.models import Sequential #This is required to make the Neural Network layer by layer
from tensorflow.keras.layers import * #Used to import LSTM layer , Dense ( fully connected layer ) , Dropout ( Regularization to prevent overfitting)
from tensorflow.keras.callbacks import ModelCheckpoint #Automatically saves the best model even if later epochs overfit or degrade
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.metrics import RootMeanSquaredError
from tensorflow.keras.optimizers import Adam #Used to adjust the model weights during training
from tensorflow.keras.models import load_model
from keras_tuner import RandomSearch

#### Below Function is called inside the train_model function

In [28]:
def build_model(hp):
    model = Sequential()
    model.add(Input(shape=(30, 6)))  # 6 input features

    model.add(LSTM(units=hp.Int('lstm_units', min_value=32, max_value=128, step=16)))
    model.add(Dropout(rate=hp.Float('dropout', 0.0, 0.5, step=0.1)))
    model.add(Dense(units=hp.Int('dense_units', 8, 64, step=8), activation='relu'))
    model.add(Dense(1, activation='sigmoid'))

    optimizer = hp.Choice('optimizer', ['adam', 'rmsprop'])
    model.compile(optimizer=optimizer,
                  loss='binary_crossentropy',
                  metrics=['accuracy'])

    return model

#### The below function is to get the ticker data for the particular ticker symbol

In [29]:
def get_ticker_data(symbol , start_date , end_date):
    buffer_start_date = start_date - timedelta(days=300)
    ticker = yf.Ticker(symbol)
    data = ticker.history(start = buffer_start_date , end = end_date)

    nifty_ticker = yf.Ticker('^NSEI')
    nifty_data = nifty_ticker.history(start = start_date , end = end_date)
    
    data['Return'] = data['Close'].pct_change()
    data['MA_50'] = data['Close'].rolling(window=50).mean()
    data['MA_200'] = data['Close'].rolling(window=200).mean()
    data['RSI'] = RSIIndicator(close=data['Close'],window=14).rsi()
    data['Volatility'] = data['Return'].rolling(window=14).std()
    
    data = data.loc[pd.to_datetime(start_date).tz_localize('UTC'):]
    nifty_data['nifty_returns'] = nifty_data['Close'].pct_change()
    data = data.join(nifty_data['nifty_returns'])
    
    data[['Volatility','MA_50','MA_200','RSI']] = scaler.fit_transform(data[['Volatility','MA_50','MA_200','RSI']])
    return data[['Return','Volatility','MA_50','MA_200','RSI','nifty_returns']].dropna()

#### The Below function is just to convert a dataframe to a numpy array so it can be manipulated more easily

In [30]:
def dataframe_to_np(X , y):
    X_np = np.array([df.values for df in X])
    y_np = np.array(y)
    return X_np , y_np

#### The Below function is to train the model

In [31]:
def train_model(symbol, train_input, train_label, validation_input, validation_labels):
    print(f'Searching for best hyper-params for {symbol} model')
    tuner = RandomSearch(
        build_model,
        objective='val_accuracy',
        max_trials=20,
        executions_per_trial=1,
        directory='lstm_tuning',
        project_name=f'nifty_predict_{symbol}'  # make it per-symbol to avoid overwrite
    )

    # Run hyperparameter search
    tuner.search(train_input, train_label,
                 validation_data=(validation_input, validation_labels),
                 epochs=10)

    # Get best model
    model1 = tuner.get_best_models(num_models=1)[0]

    print(f'Hyper-Params for best model of {symbol} have been found')
    # Save best weights
    cp = ModelCheckpoint(f'model1_{symbol}/model.keras', save_best_only=True)
    model1.fit(train_input, train_label,
               validation_data=(validation_input, validation_labels),
               epochs=10, callbacks=[cp])

    # Reload best version and evaluate
    model1 = load_model(f'model1_{symbol}/model.keras')
    val_predictions = model1.predict(validation_input).flatten()
    predicted_labels = (val_predictions > 0.5).astype(int)
    accuracy = np.mean(predicted_labels == validation_labels)

    return accuracy


#### Below we call all the required functions and store the accuracies for each model in the respective dictionary
#### Note that I have excluded a few companies while going through the tickers as they were having a problem while executing the code

In [32]:
today = datetime.today().date()
base_date = today - timedelta(days=180)
validation_base_date = today - timedelta(days=70)
accuracy_dict = {}
tickers = nifty50_tickers[:13] + nifty50_tickers[15:40] + nifty50_tickers[41:]

for symbol in tickers:
    print('Entered the Loop for',symbol)
    
    # Initialize training window
    start_date = base_date
    end_date = start_date + timedelta(days=51)

    # Initialize validation window
    validation_start_date = validation_base_date
    validation_end_date = validation_start_date + timedelta(days=51)

    # Create validation dataset
    validation_input = []
    validation_labels = []
    while validation_end_date < today:
        validation_iteration_data = get_ticker_data(symbol, validation_start_date, validation_end_date)

        if len(validation_iteration_data) < 31:
            validation_start_date += timedelta(days=1)
            validation_end_date += timedelta(days=1)
            continue

        label = int(validation_iteration_data['Return'].iloc[-1] > 0)
        validation_iteration_data = validation_iteration_data.iloc[:-1].tail(30)

        if len(validation_iteration_data) < 30:
            validation_start_date += timedelta(days=1)
            validation_end_date += timedelta(days=1)
            continue
        
        validation_input.append(validation_iteration_data)
        validation_labels.append(label)

        validation_start_date += timedelta(days=1)
        validation_end_date += timedelta(days=1)

    validation_input, validation_labels = dataframe_to_np(validation_input, validation_labels)

    print('Validation Data Collected for', symbol)
    
    # Create training dataset
    input_data = []
    output_labels = []
    while end_date < validation_base_date:
        data = get_ticker_data(symbol, start_date, end_date)

        if len(data) < 30:
            start_date += timedelta(days=1)
            end_date += timedelta(days=1)
            continue

        label = int(data['Return'].iloc[-1] > 0)
        data = data.iloc[:-1].tail(30)

        if len(data) < 30:
            start_date += timedelta(days=1)
            end_date += timedelta(days=1)
            continue

        input_data.append(data)
        output_labels.append(label)

        start_date += timedelta(days=1)
        end_date += timedelta(days=1)

    input_data, output_labels = dataframe_to_np(input_data, output_labels)
    # Build and train model
    
    print('Training Data Collected for',symbol)
    
    accuracy = train_model(symbol ,input_data , output_labels , validation_input , validation_labels)
    accuracy_dict[symbol] = accuracy
    
    print('Loop done for',symbol)

print('Execution Done ----- XXXX -----')

Entered the Loop for ADANIENT.NS
Validation Data Collected for ADANIENT.NS
Training Data Collected for ADANIENT.NS
Searching for best hyper-params for ADANIENT.NS model
Reloading Tuner from lstm_tuning/nifty_predict_ADANIENT.NS/tuner0.json
Hyper-Params for best model of ADANIENT.NS have been found
Epoch 1/10


/opt/anaconda3/lib/python3.12/site-packages/keras/src/saving/saving_lib.py:802: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 16 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 89ms/step - accuracy: 0.4907 - loss: 0.6914 - val_accuracy: 0.6316 - val_loss: 0.6771
Epoch 2/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - accuracy: 0.5679 - loss: 0.6822 - val_accuracy: 0.6316 - val_loss: 0.6752
Epoch 3/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - accuracy: 0.6900 - loss: 0.6616 - val_accuracy: 0.6316 - val_loss: 0.6727
Epoch 4/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - accuracy: 0.6128 - loss: 0.6609 - val_accuracy: 0.6316 - val_loss: 0.6701
Epoch 5/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - accuracy: 0.7317 - loss: 0.6462 - val_accuracy: 0.6316 - val_loss: 0.6672
Epoch 6/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - accuracy: 0.7223 - loss: 0.6230 - val_accuracy: 0.6316 - val_loss: 0.6643
Epoch 7/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - accuracy: 0.7015 - loss: 0.6239 - val_accuracy: 0.6316 - val_loss: 0.6612
Epoch 8/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - accuracy: 0.7453 - loss: 0.6076 - val_accuracy: 0.6316 - val_loss: 0.6580
Epoch 9/10


/opt/anaconda3/lib/python3.12/site-packages/keras/src/saving/saving_lib.py:802: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 16 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 91ms/step - accuracy: 0.5323 - loss: 0.6899 - val_accuracy: 0.5789 - val_loss: 0.6798
Epoch 2/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - accuracy: 0.6117 - loss: 0.6950 - val_accuracy: 0.5789 - val_loss: 0.6793
Epoch 3/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - accuracy: 0.5427 - loss: 0.6910 - val_accuracy: 0.5789 - val_loss: 0.6788
Epoch 4/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - accuracy: 0.6336 - loss: 0.6881 - val_accuracy: 0.5789 - val_loss: 0.6781
Epoch 5/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 0.5241 - loss: 0.6890 - val_accuracy: 0.5789 - val_loss: 0.6783
Epoch 6/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 0.6419 - loss: 0.6776 - val_accuracy: 0.5789 - val_loss: 0.6786
Epoch 7/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 0.4896 - loss: 0.6928 - val_accuracy: 0.5789 - val_loss: 0.6787
Epoch 8/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 0.6325 - loss: 0.6765 - val_accuracy: 0.5789 - val_loss: 0.6788
Epoch 9/10


...

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 88ms/step - accuracy: 0.6325 - loss: 0.6736 - val_accuracy: 0.5263 - val_loss: 0.6965
Epoch 2/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - accuracy: 0.6117 - loss: 0.6757 - val_accuracy: 0.5263 - val_loss: 0.6950
Epoch 3/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - accuracy: 0.6221 - loss: 0.6472 - val_accuracy: 0.5263 - val_loss: 0.6956
Epoch 4/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - accuracy: 0.6117 - loss: 0.6548 - val_accuracy: 0.4737 - val_loss: 0.6989
Epoch 5/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - accuracy: 0.5805 - loss: 0.6729 - val_accuracy: 0.5263 - val_loss: 0.6948
Epoch 6/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - accuracy: 0.6221 - loss: 0.6536 - val_accuracy: 0.5263 - val_loss: 0.6937
Epoch 7/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 0.6430 - loss: 0.6567 - val_accuracy: 0.5263 - val_loss: 0.6943
Epoch 8/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - accuracy: 0.6325 - loss: 0.6341 - val_accuracy: 0.5263 - val_loss: 0.6979
Epoch 9/10


/opt/anaconda3/lib/python3.12/site-packages/keras/src/saving/saving_lib.py:802: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 2 variables whereas the saved optimizer has 9 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 92ms/step - accuracy: 0.4353 - loss: 0.7023 - val_accuracy: 0.2105 - val_loss: 0.7466
Epoch 2/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - accuracy: 0.5542 - loss: 0.6903 - val_accuracy: 0.2105 - val_loss: 0.7342
Epoch 3/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - accuracy: 0.5783 - loss: 0.6811 - val_accuracy: 0.2105 - val_loss: 0.7377
Epoch 4/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - accuracy: 0.5772 - loss: 0.6878 - val_accuracy: 0.3684 - val_loss: 0.7309
Epoch 5/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - accuracy: 0.6670 - loss: 0.6597 - val_accuracy: 0.3158 - val_loss: 0.7259
Epoch 6/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - accuracy: 0.6440 - loss: 0.6484 - val_accuracy: 0.2632 - val_loss: 0.7252
Epoch 7/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - accuracy: 0.6128 - loss: 0.6767 - val_accuracy: 0.2632 - val_loss: 0.7207
Epoch 8/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - accuracy: 0.6117 - loss: 0.6748 - val_accuracy: 0.2105 - val_loss: 0.7156
Epoch 9/10


/opt/anaconda3/lib/python3.12/site-packages/keras/src/saving/saving_lib.py:802: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 2 variables whereas the saved optimizer has 9 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 87ms/step - accuracy: 0.5345 - loss: 0.6960 - val_accuracy: 0.2105 - val_loss: 0.7524
Epoch 2/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 0.6555 - loss: 0.6610 - val_accuracy: 0.2105 - val_loss: 0.7721
Epoch 3/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 0.6336 - loss: 0.6492 - val_accuracy: 0.2105 - val_loss: 0.7626
Epoch 4/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 0.6128 - loss: 0.6527 - val_accuracy: 0.2105 - val_loss: 0.7638
Epoch 5/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - accuracy: 0.6440 - loss: 0.6303 - val_accuracy: 0.2632 - val_loss: 0.7351
Epoch 6/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 0.6232 - loss: 0.6373 - val_accuracy: 0.4211 - val_loss: 0.7357
Epoch 7/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - accuracy: 0.6336 - loss: 0.6286 - val_accuracy: 0.5263 - val_loss: 0.7279
Epoch 8/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - accuracy: 0.6128 - loss: 0.6171 - val_accuracy: 0.5263 - val_loss: 0.7133
Epoch 9/10


#### Below is the piece of code to sort the obtained accuracies from above

In [33]:
sorted_accuracies = sorted(accuracy_dict.items(), key=lambda x: x[1], reverse=True)

for symbol, accuracy in sorted_accuracies:
    print(f"{symbol}: {accuracy:.2%}")

TITAN.NS: 89.47%
HCLTECH.NS: 84.21%
ITC.NS: 84.21%
NESTLEIND.NS: 78.95%
SBILIFE.NS: 78.95%
BEL.NS: 73.68%
COALINDIA.NS: 73.68%
HDFCLIFE.NS: 73.68%
M&M.NS: 73.68%
POWERGRID.NS: 73.68%
TATACONSUM.NS: 73.68%
TATAMOTORS.NS: 73.68%
TECHM.NS: 73.68%
MAXHEALTH.NS: 68.42%
WIPRO.NS: 68.42%
ADANIENT.NS: 63.16%
AXISBANK.NS: 63.16%
CIPLA.NS: 63.16%
INDIGO.NS: 63.16%
TATASTEEL.NS: 63.16%
ADANIPORTS.NS: 57.89%
APOLLOHOSP.NS: 57.89%
BAJAJ-AUTO.NS: 57.89%
BAJAJFINSV.NS: 57.89%
KOTAKBANK.NS: 57.89%
TCS.NS: 57.89%
GRASIM.NS: 52.63%
HINDALCO.NS: 52.63%
ONGC.NS: 52.63%
SBIN.NS: 52.63%
TRENT.NS: 52.63%
ASIANPAINT.NS: 47.37%
BAJFINANCE.NS: 47.37%
RELIANCE.NS: 47.37%
JSWSTEEL.NS: 42.11%
NTPC.NS: 42.11%
SHRIRAMFIN.NS: 42.11%
DRREDDY.NS: 36.84%
JIOFIN.NS: 36.84%
BHARTIARTL.NS: 31.58%
ICICIBANK.NS: 31.58%
INFY.NS: 31.58%
HINDUNILVR.NS: 26.32%
LT.NS: 26.32%
ULTRACEMCO.NS: 26.32%
HDFCBANK.NS: 15.79%
MARUTI.NS: 15.79%


#### Now we only choose those companies for which we have a prediction accuracy > threshold, for this project, i have decided to keep a threshold of 60%

In [34]:
threshold = 0.6
high_accuracy_symbols = {
    symbol: acc for symbol, acc in accuracy_dict.items() if acc > threshold
}

print(high_accuracy_symbols)

{'ADANIENT.NS': 0.631578947368421, 'AXISBANK.NS': 0.631578947368421, 'BEL.NS': 0.7368421052631579, 'CIPLA.NS': 0.631578947368421, 'COALINDIA.NS': 0.7368421052631579, 'HCLTECH.NS': 0.8421052631578947, 'HDFCLIFE.NS': 0.7368421052631579, 'INDIGO.NS': 0.631578947368421, 'ITC.NS': 0.8421052631578947, 'M&M.NS': 0.7368421052631579, 'MAXHEALTH.NS': 0.6842105263157895, 'NESTLEIND.NS': 0.7894736842105263, 'POWERGRID.NS': 0.7368421052631579, 'SBILIFE.NS': 0.7894736842105263, 'TATACONSUM.NS': 0.7368421052631579, 'TATAMOTORS.NS': 0.7368421052631579, 'TATASTEEL.NS': 0.631578947368421, 'TECHM.NS': 0.7368421052631579, 'TITAN.NS': 0.8947368421052632, 'WIPRO.NS': 0.6842105263157895}


#### Below is an extra piece of code where i can just run it to check how accurate have I been today
#### The below mentioned function is to get the return today and to check if my predicton is correct or not
#### Note, call this function only after the trading day is over (preferably after some time its been over, depending on how fast yfinance updates its data) otherwise it will give return of yesterday

In [35]:
def get_today_return(symbol):
    ticker = yf.Ticker(symbol)
    today_data = ticker.history(period='2d').Close
    today_return = (today_data[-1] / today_data[0]) - 1
    
    return today_return

#### Below i check the return and compare it to my prediction

In [36]:
today = datetime.today().date()
start_date = today - timedelta(days=50)
end_date = today - timedelta(days=1)
correct = 0
total = 0

total_return = 0
for symbol in high_accuracy_symbols :
    total+=1
    data = get_ticker_data(symbol, start_date, end_date)

    if len(data) < 30:
            print(f'Not Sufficient data for {symbol}')
            continue
    
    data = data.tail(30)
    input_np = np.array([data.values])
    
    model = load_model(f'model1_{symbol}/model.keras')
    
    prediction = model.predict(input_np)[0][0]
    actual = get_today_return(symbol)
    if int(actual > 0) == int(prediction > 0.5):
        print(f'Prediction correct for {symbol}')
        correct+=1
        
    else:
        print(f'Prediction Wrong for {symbol}')
        
    total_return+=actual 
print('Our Accuracy for today is',correct / total)
print('Our Total correct predictions are',correct)
print('Total Companies in our Portfolio are',total)
print('Our Total Return for today is',total_return)
print('Predictions Done')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
Prediction correct for ADANIENT.NS


/var/folders/zp/_z5ytlrj39x9_lyt_g51rrfm0000gn/T/ipykernel_10034/92847729.py:4: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  today_return = (today_data[-1] / today_data[0]) - 1


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
Prediction Wrong for AXISBANK.NS


/var/folders/zp/_z5ytlrj39x9_lyt_g51rrfm0000gn/T/ipykernel_10034/92847729.py:4: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  today_return = (today_data[-1] / today_data[0]) - 1


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
Prediction correct for BEL.NS


...

/var/folders/zp/_z5ytlrj39x9_lyt_g51rrfm0000gn/T/ipykernel_10034/92847729.py:4: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  today_return = (today_data[-1] / today_data[0]) - 1


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
Prediction correct for TITAN.NS


/var/folders/zp/_z5ytlrj39x9_lyt_g51rrfm0000gn/T/ipykernel_10034/92847729.py:4: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  today_return = (today_data[-1] / today_data[0]) - 1


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
Prediction Wrong for WIPRO.NS
Our Accuracy for today is 0.8
Our Total correct predictions are 16
Total Companies in our Portfolio are 20
Our Total Return for today is 0.0
Predictions Done


/var/folders/zp/_z5ytlrj39x9_lyt_g51rrfm0000gn/T/ipykernel_10034/92847729.py:4: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  today_return = (today_data[-1] / today_data[0]) - 1
